In [1]:
import torch
from torchvision import datasets, transforms


In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.CenterCrop((256, 256))])
images = datasets.ImageFolder("./data/images")